# core

> Fill in a module description here

In [ ]:
#| default_exp core

In [ ]:
#| hide 

from nbdev.showdoc import show_doc

In [ ]:
#| exports 

from humble_database.database import Database
from humble_database.data_model import DataModel,Query

ImportError: dlopen(/Users/schlinkertc/opt/anaconda3/envs/humble-database/lib/python3.11/site-packages/cryptography/hazmat/bindings/_rust.abi3.so, 0x0002): tried: '/Users/schlinkertc/opt/anaconda3/envs/humble-database/lib/python3.11/site-packages/cryptography/hazmat/bindings/_rust.abi3.so' (mach-o file, but is an incompatible architecture (have 'arm64', need 'x86_64')), '/System/Volumes/Preboot/Cryptexes/OS/Users/schlinkertc/opt/anaconda3/envs/humble-database/lib/python3.11/site-packages/cryptography/hazmat/bindings/_rust.abi3.so' (no such file), '/Users/schlinkertc/opt/anaconda3/envs/humble-database/lib/python3.11/site-packages/cryptography/hazmat/bindings/_rust.abi3.so' (mach-o file, but is an incompatible architecture (have 'arm64', need 'x86_64'))

In [ ]:
show_doc(Database)

In [ ]:
show_doc(DataModel)

In [ ]:
show_doc(Query)

In [ ]:
#| hide 
import nbdev; nbdev.nbdev_export()